# Cargar librerías y datsets

In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk 
import seaborn as sns
import spacy
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import mean_squared_error, accuracy_score, classification_report
from sklearn.feature_extraction.text import CountVectorizer
import re
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.pipeline import make_pipeline
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [32]:
train_data = pd.read_json("train_financial.json")
test_d = pd.read_json("FINANCIAL_TEST_LIMPIO.json")

In [33]:
df = train_data
test_data = test_d

# Vader

Dado que VADER es un algoritmo basado en reglas y no un modelo entrenable, su precisión y desempeño pueden ser evaluados aplicándolo directamente sobre df

In [34]:
def apply_vader_and_label(dataframe):
    analyzer = SentimentIntensityAnalyzer()
    
    # Aplicar el análisis de sentimientos a cada texto
    dataframe[['neg', 'neu', 'pos', 'compound']] = dataframe['text'].apply(lambda x: pd.Series(analyzer.polarity_scores(x)))
    
    # Función para determinar la etiqueta basada en el puntaje compound
    def get_sentiment_label(compound_score):
        if compound_score >= 0.05:
            return 2  # positivo
        elif compound_score <= -0.05:
            return 0  # negativo
        else:
            return 1  # neutro
    
    # Aplicar la función a la columna compound para obtener las etiquetas predichas
    dataframe['predicted_label'] = dataframe['compound'].apply(get_sentiment_label)
    return dataframe

# Aplicar la función al DataFrame de entrenamiento
df = apply_vader_and_label(df)


# Mostrar los DataFrames actualizados
print("DataFrame de entrenamiento:")
print(df)


DataFrame de entrenamiento:
        label                                               text  \
131712      0  many commercial vessel got stuck narrow bay bo...   
131713      2        diluted loss per share stood eur versus eur   
131714      1  broad range connectivity option including hspa...   
131715      2  however biohit estimate total net sale continu...   
131716      1  mr bakman see also expansion option link claim...   
...       ...                                                ...   
135080      0  operating profit excluding item totaled eur mn...   
135081      1  start negotiation relating glaston efficiency ...   
135082      1  company exported twenty european country well ...   
135083      1  started software capable retrieving data typed...   
135084      1  margin call zanadvorov given chance make purch...   

                     Dataset    neg    neu    pos  compound  predicted_label  
131712  Financial Phrasebank  0.143  0.857  0.000   -0.2500                0

# Metricas de rendimiento (informe clasificación)

VADER no es un modelo de aprendizaje supervisado que requiere un proceso de entrenamiento. Es un modelo de análisis de sentimientos basado en reglas y lexicón, lo que significa que no se entrena en un conjunto de datos antes de aplicarse a otros datos. Su funcionamiento se basa en un diccionario de palabras con puntajes predefinidos y reglas para calcular la polaridad del sentimiento.

In [35]:
# Generar el informe de clasificación para el conjunto de entrenamiento
train_report = classification_report(df['label'], df['predicted_label'], output_dict=True)
train_report_df = pd.DataFrame(train_report).transpose()
print("Informe de clasificación para el conjunto de entrenamiento")
print(train_report_df)

Informe de clasificación para el conjunto de entrenamiento
              precision    recall  f1-score      support
0              0.426966  0.273381  0.333333   417.000000
1              0.759804  0.532646  0.626263  2037.000000
2              0.392133  0.715996  0.506739   919.000000
accuracy       0.550548  0.550548  0.550548     0.550548
macro avg      0.526301  0.507341  0.488778  3373.000000
weighted avg   0.618481  0.550548  0.557483  3373.000000



El uso de VADER para el análisis de sentimientos no se presta directamente a técnicas de validación cruzada (cross-validation) típicas del aprendizaje supervisado, ya que VADER es un algoritmo basado en reglas y lexicón, no un modelo que se entrena con datos específicos